# Scope: Trecerea de la problema la solutie

In [1]:
import pandas as pd
import numpy as np
from dotenv import load_dotenv
import os
import openai
import json
# Helps asyncio run within Jupyter
import nest_asyncio

import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored


load_dotenv()
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

if os.getenv("OPENAI_API_KEY") is not None:
    print ("OPENAI_API_KEY is ready")
else:
    print ("OPENAI_API_KEY environment variable not found")

GPT_MODEL = "gpt-3.5-turbo-16k"

OPENAI_API_KEY is ready


In [2]:
@retry(wait=wait_random_exponential(min=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, temperature=0, model=GPT_MODEL):
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + openai.api_key,
    }
    json_data = {"model": model, "messages": messages, "temperature": temperature}
    if functions is not None:
        json_data.update({"functions": functions})
    if function_call is not None:
        json_data.update({"function_call": function_call})
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        return response
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [3]:
# Read data about the product

with open('/Users/vladbordei/Documents/Development/ProductExplorer/data/interim/short_product_information.json') as file:
    json_string = file.read()
    general_product_data = json.loads(json_string)

In [4]:
#asin_list_path = './data/external/asin_list.csv'
asin_list_path = '/Users/vladbordei/Documents/Development/ProductExplorer/data/external/asin_list.csv'
asin_list = pd.read_csv(asin_list_path)['asin'].tolist()

In [5]:
attribute_clusters_with_percentage_path = '/Users/vladbordei/Documents/Development/ProductExplorer/data/interim/attribute_clusters_with_percentage.csv'
attributes = pd.read_csv(attribute_clusters_with_percentage_path)

In [6]:
reviews_with_clusters_path = '/Users/vladbordei/Documents/Development/ProductExplorer/data/interim/reviews_with_clusters.csv'
reviews_with_clusters = pd.read_csv(reviews_with_clusters_path)

In [7]:
df_problem_statements_path = '/Users/vladbordei/Documents/Development/ProductExplorer/data/interim/problem_statements.csv'
df_problem_statements = pd.read_csv(df_problem_statements_path)

In [8]:
attributes_data = attributes[['Attribute', 'cluster_label', 'positive_sentiment',
       'negative_sentiment','observation_count', 'rating_avg', 'attribute_percentage']]

In [9]:
general_product_data

{'Product Summary': 'The Raised Garden Bed is a 6x3x2ft galvanized planter box designed for growing vegetables, flowers, and herbs. It is made of stable thickened steel with an evaluated coating paint, ensuring long-term use without rusting. The open bottom design allows for good drainage, promoting healthy plant growth. Setting up the garden bed is quick and easy, as it does not require any tools. The butterfly screws provided can be fastened by hand in just 5-10 minutes. The garden bed also features a cured edge for safety, preventing any injuries to your hands.',
 'In_the_Box': 'The package includes the FRIZIONE Galvanized Metal Raised Garden Bed and all the necessary hardware for assembly.',
 'technical_facts': 'The garden bed is made of galvanized steel with an anti-rust coating. It has a 2ft depth and measures 6x3ft in size.',
 'features': '1. Size: 6x3x2ft, providing ample growing space for vegetables, herbs, and flowers.\n2. Made of stable thickened steel with evaluated coating

In [10]:
product_improvement_function = [
    {
        "name": "product_improvement_function",
        "description": "This function is designed to provide engineering solutions to address the primary issues with a product. The function uses the data from customer complaints and the product's description to propose technical product improvements. The Implementation Details  should be concise, yet comprehensive, explaining the rationale behind the solution and step-by-step instructions for carrying out the implementation. It should not contain jargon or technical terms that are not commonly understood by engineers in the relevant field.",
        "parameters": {
            "type": "object",
            "properties": {
                "Product Improvement 1": {
                    "type": "object",
                    "properties": {
                        "Title": {
                            "type": "string",
                            "description": "The title or short description of the first proposed product improvement."
                        },
                        "Implementation Details for the engineer": {
                            "type": "string",
                            "description": "A detailed, 200-word description of the first improvement solution, including specific instructions for implementation by an engineer."
                        },
                        "Considerations": {
                            "type": "string",
                            "description": "Considerations and potential challenges in the implementation of the proposed solution."
                        }
                    },
                    "required": ["Title", "Implementation Details for the engineer", "Considerations"]
                },
                "Product Improvement 2": {
                    "type": "object",
                    "properties": {
                        "Title": {
                            "type": "string",
                            "description": "The title or short description of the second proposed product improvement."
                        },
                        "Implementation Details for the engineer": {
                            "type": "string",
                            "description": "A detailed, 200-word description of the second improvement solution, including specific instructions for implementation by an engineer."
                        },
                        "Considerations": {
                            "type": "string",
                            "description": "Considerations and potential challenges in the implementation of the proposed solution."
                        }
                    },
                    "required": ["Title", "Implementation Details for the engineer", "Considerations"]
                },
                "Product Improvement 3": {
                    "type": "object",
                    "properties": {
                        "Title": {
                            "type": "string",
                            "description": "The title or short description of the third proposed product improvement."
                        },
                        "Implementation Details for the engineer": {
                            "type": "string",
                            "description": "AA detailed, 200-word description of the third improvement solution, including specific instructions for implementation by an engineer."
                        },
                        "Considerations": {
                            "type": "string",
                            "description": "Considerations and potential challenges in the implementation of the proposed solution."
                        }
                    },
                    "required": ["Title", "Implementation Details for the engineer", "Considerations"]
                },
                "Product Improvement 4": {
                    "type": "object",
                    "properties": {
                        "Title": {
                            "type": "string",
                            "description": "The title or short description of the fourth proposed product improvement."
                        },
                        "Implementation Details for the engineer": {
                            "type": "string",
                            "description": "A detailed, 200-word description of the fourth improvement solution, including specific instructions for implementation by an engineer."
                        },
                        "Considerations": {
                            "type": "string",
                            "description": "Considerations and potential challenges in the implementation of the proposed solution."
                        }
                    },
                    "required": ["Title", "Implementation Details for the engineer", "Considerations"]
                },
                "Product Improvement 5": {
                    "type": "object",
                    "properties": {
                        "Title": {
                            "type": "string",
                            "description": "The title or short description of the fifth proposed product improvement."
                        },
                        "Implementation Details for the engineer": {
                            "type": "string",
                            "description": "A detailed, 200-word description of the fifth improvement solution, including specific instructions for implementation by an engineer."
                        },
                        "Considerations": {
                            "type": "string",
                            "description": "Considerations and potential challenges in the implementation of the proposed solution."
                        }
                    },
                    "required": ["Title", "Implementation Details for the engineer", "Considerations"]
                },
                "Product Improvement 6": {
                    "type": "object",
                    "properties": {
                        "Title": {
                            "type": "string",
                            "description": "The title or short description of the sixth proposed product improvement."
                        },
                        "Implementation Details for the engineer": {
                            "type": "string",
                            "description": "A detailed, 200-word description of the sixth improvement solution, including specific instructions for implementation by an engineer."
                        },
                        "Considerations": {
                            "type": "string",
                            "description": "Considerations and potential challenges in the implementation of the proposed solution."
                        }
                    },
                    "required": ["Title", "Implementation Details for the engineer", "Considerations"]
                },
                "Product Improvement 7": {
                    "type": "object",
                    "properties": {
                        "Title": {
                            "type": "string",
                            "description": "The title or short description of the seventh proposed product improvement."
                        },
                        "Implementation Details for the engineer": {
                            "type": "string",
                            "description": "A detailed, 200-word description of the seventh improvement solution, including specific instructions for implementation by an engineer."
                        },
                        "Considerations": {
                            "type": "string",
                            "description": "Considerations and potential challenges in the implementation of the proposed solution."
                        }
                    },
                    "required": ["Title", "Implementation Details for the engineer", "Considerations"]
                },
            },
            "required": ["Product Improvement 1", "Product Improvement 2", "Product Improvement 3", "Product Improvement 4", "Product Improvement 5", "Product Improvement 6", "Product Improvement 7"]
        }
    }
]



In [122]:
problem = df_problem_statements['problem_statement']
messages = [
    {"role": "user", "content": f"PROBLEM STATEMENT: {problem} ,PRODUCT DESCRIPTION: {general_product_data} AND ANALYSIS: {attributes_data}"}
]

# Send the request to the LLM and get the response
response =  chat_completion_request(
    messages=messages,
    functions= product_improvement_function,
    function_call={"name": "product_improvement_function"},
    temperature=0.5,
    model=GPT_MODEL
)

In [120]:
response.json()

{'id': 'chatcmpl-7fuuNBKveJnm6LcutZ7ybWpEFVa4r',
 'object': 'chat.completion',
 'created': 1690224559,
 'model': 'gpt-3.5-turbo-16k-0613',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': None,
    'function_call': {'name': 'product_improvement_function',
     'arguments': '{\n  "Product Improvement 1": {\n    "Title": "Increase Durability",\n    "Implementation Details for the engineer": "1. Use a thicker gauge galvanized steel for the construction of the garden bed to increase its strength and durability.\\n2. Apply multiple layers of anti-rust coating to ensure long-term use without rusting.\\n3. Reinforce the corners and edges of the garden bed with additional support brackets or welds.\\n4. Use high-quality butterfly screws that are more resistant to wear and tear.\\n5. Conduct rigorous testing to ensure that the garden bed can withstand heavy loads and harsh weather conditions.",\n    "Considerations": "1. The thicker gauge steel and additional support

In [123]:
# Process the response and store in the dictionary
chat_response = response.json()["choices"][0]
print(chat_response)

{'index': 0, 'message': {'role': 'assistant', 'content': None, 'function_call': {'name': 'product_improvement_function', 'arguments': '{\n  "Product Improvement 1": {\n    "Title": "Increase Durability",\n    "Implementation Details for the engineer": "To increase the durability of the Raised Garden Bed, the following steps can be taken:\\n1. Use a thicker gauge galvanized steel for the construction of the planter box. This will provide added strength and resistance to bending or warping.\\n2. Apply a double layer of anti-rust coating to the steel. This will enhance the protective barrier against corrosion and extend the lifespan of the garden bed.\\n3. Reinforce the corners and edges of the planter box with additional welding or bracing. This will prevent any weak points and improve overall structural integrity.\\n4. Use high-quality butterfly screws for assembly. These screws should be made of durable materials, such as stainless steel, to ensure they do not rust or degrade over time

In [124]:
data = eval(chat_response['message']['function_call']['arguments'])

In [125]:
data

{'Product Improvement 1': {'Title': 'Increase Durability',
  'Implementation Details for the engineer': 'To increase the durability of the Raised Garden Bed, the following steps can be taken:\n1. Use a thicker gauge galvanized steel for the construction of the planter box. This will provide added strength and resistance to bending or warping.\n2. Apply a double layer of anti-rust coating to the steel. This will enhance the protective barrier against corrosion and extend the lifespan of the garden bed.\n3. Reinforce the corners and edges of the planter box with additional welding or bracing. This will prevent any weak points and improve overall structural integrity.\n4. Use high-quality butterfly screws for assembly. These screws should be made of durable materials, such as stainless steel, to ensure they do not rust or degrade over time.\n5. Conduct thorough quality control checks during the manufacturing process to identify and address any potential weaknesses or defects.\n6. Provide 

In [129]:
set(attributes_data['Attribute'])


{'Buyer Motivation',
 'Colors',
 'Customer Expectations',
 'Danger Appraisal',
 'Design and Appearance',
 'Durability',
 'Ease of Use',
 'How the product is used',
 'Issues',
 'Noise and Smell',
 'Packaging',
 'Parts and Components',
 'Price',
 'Quality',
 'Season',
 'Setup and Instructions',
 'Size and Fit',
 'User Description',
 'When the product is used',
 'Where the product is used'}

In [131]:
attributes_data[attributes_data['Attribute'] == 'Buyer Motivation']

,Attribute,cluster_label,positive_sentiment,negative_sentiment,observation_count,rating_avg,attribute_percentage
0,Buyer Motivation,Bed replacement,0.662187,0.128551,6,4,4.615385
1,Buyer Motivation,Buyer Motivation,0.730204,0.143931,40,4,30.769231
2,Buyer Motivation,Motivation: Raised Bed Gardening,0.553676,0.240755,25,4,19.230769
3,Buyer Motivation,Motivation: To have a convenient way to garden,0.737425,0.123885,23,4,17.692308
4,Buyer Motivation,To assist my mother with her mobility limitations,0.522197,0.315513,30,4,23.076923
5,Buyer Motivation,To try out a new product,0.435807,0.343664,6,4,4.615385


In [133]:
attributes_data[attributes_data['Attribute'] == 'Colors']

,Attribute,cluster_label,positive_sentiment,negative_sentiment,observation_count,rating_avg,attribute_percentage
6,Colors,Colors,0.833212,0.040563,10,5,45.454545
7,Colors,Limited color options,0.638209,0.221642,6,4,27.272727
8,Colors,Unknown,0.320234,0.641082,3,3,13.636364
9,Colors,Variety of colors,0.688907,0.056164,3,4,13.636364


In [11]:
attributes_data['observation_count'].sum()

1541

In [18]:
number_of_reviews = reviews_with_clusters['URL'].unique().shape[0]
number_of_reviews

220

In [19]:
attributes_data.rename(columns={'attribute_percentage': 'percentage_of_observations_vs_total_number_per_attribute'}, inplace=True)
attributes_data['percentage_of_observations_vs_total_number_of_reviews'] = attributes_data['observation_count'] / number_of_reviews

/var/folders/f6/0jn5yf2x1b319g8j96pnjbwh0000gn/T/ipykernel_17207/488290426.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  attributes_data['percentage_of_observations_vs_total_number_of_reviews'] = attributes_data['observation_count'] / number_of_reviews


In [24]:
attributes_data.sort_values(by='percentage_of_observations_vs_total_number_of_reviews', ascending=False)

,Attribute,cluster_label,positive_sentiment,negative_sentiment,observation_count,rating_avg,percentage_of_observations_vs_total_number_per_attribute,percentage_of_observations_vs_total_number_of_reviews
27,Ease of Use,Ease of Use,0.781626,0.104111,80,5,86.956522,0.363636
10,Customer Expectations,Customer Expectations,0.572575,0.258386,60,4,46.511628,0.272727
67,Quality,Quality,0.766979,0.117326,54,4,64.285714,0.245455
24,Durability,Durability,0.785929,0.102252,51,4,83.606557,0.231818
21,Design and Appearance,Design and Appearance,0.753909,0.113694,50,5,86.206897,0.227273
...,...,...,...,...,...,...,...,...
14,Danger Appraisal,No specific observations on the safety mentioned,0.671206,0.074277,1,4,3.703704,0.004545
17,Danger Appraisal,Safe for children,0.990170,0.002611,1,5,3.703704,0.004545
42,Noise and Smell,Creates a soothing sound,0.990017,0.005117,1,5,3.846154,0.004545
46,Noise and Smell,No specific observations on the noise level or...,0.671206,0.074277,1,4,3.846154,0.004545
